In [1]:
from typing import Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm

epochs = 2
batch_size = 128
classes_count = 10

# Define a transform to normalize the data
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ],
)

# Load the training and test datasets
trainset = torchvision.datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=transform,
)
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=1,
)

testset = torchvision.datasets.CIFAR10(
    root="./data",
    train=False,
    download=True,
    transform=transform,
)
testloader = torch.utils.data.DataLoader(
    testset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=1,
)


# Define the neural network
class Net(nn.Module):
    def __init__(self, input_size: Tuple[int, int], classes_count: int):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 16, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        input_flatten_size = int(16 * (input_size[0] / 4) * (input_size[1] / 4))
        self.fc1 = nn.Linear(input_flatten_size, 64)
        self.fc2 = nn.Linear(64, classes_count)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


for data in trainloader:
    input_data, _ = data
    input_size = input_data.shape

net = Net(input_size[2:4], classes_count)

# Define a loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

# Train the network
for epoch in range(epochs):
    progress_bar = tqdm(trainloader, position=0, leave=True)
    running_loss = 0.0
    for data in trainloader:
        input_data, labels = data

        optimizer.zero_grad()

        outputs = net(input_data)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        progress_bar.set_description(f"Epoch {epoch+1}/{epochs}")
        progress_bar.set_postfix(train_loss=round(loss.item(), 3))
        progress_bar.update()


Files already downloaded and verified
Files already downloaded and verified


Epoch 2/2: 100%|█████████▉| 390/391 [00:59<00:00,  5.38it/s, train_loss=1.57]

In [12]:
# Test the model
predictions = []
targets = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        predictions.append(outputs)
        targets.append(labels)

predictions = torch.argmax(torch.cat(predictions, dim=0), dim=1)
targets = torch.cat(targets, dim=0)

In [14]:
from sklearn.metrics import accuracy_score

print("Test  Accuracy : {:.3f}".format(accuracy_score(targets, predictions)))

Test  Accuracy : 0.431
